## 分类问题笔记 2018-06-11

### 读取数据

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

In [ ]:
X, y = mnist['data'], mnist['target']
print(X.shape)
print(y.shape)

In [ ]:
# 查看数据集中的某一个数字
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
some_digit = X[36000]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap=matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
y[36000]

### 创建测试集

In [ ]:
# 训练集：前60000， 测试集：后10000
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# 打乱训练集
import numpy as np
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

### 训练一个二分类器

In [ ]:
y_train_5 = (y_train == 5) #识别5或非5
y_test_5 = (y_test == 5)

from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

# 查出数字5的图片
sgd_clf.predict([some_digit])

### 对性能的评估

#### 1. 使用交叉验证测量准确性

In [ ]:
from sklearn.model_selection import StratifiedKFold # 分层采样交叉验证
from sklearn.base import clone

skfolds = StratifiedKFold(n_split=3, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_test_5[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred)) # 0.9502, 0.96565, 0.96495


In [ ]:
# 使用cross_val_score()函数评估SGDClassifier模型
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

In [ ]:
# 使用一个很笨的分类器查看性能
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(x), 1), dtype=bool)
    
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

#### 2. 混淆矩阵

In [ ]:
# 使用cross_val_predict得到干净的预测
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

# 使用混淆矩阵函数
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

In [ ]:
# 完美分类器的混淆矩阵
confusion_matrix(y_train_5, y_train_perfect_predictions)

#### 3. 准确率与召回率

*正例预测的精度*
* 准确率(precision) = TP / (TP + FP) 

*正例被探测出的比率*

* 召回率(recall) = TP / (TP + FN)

*准确率与召回率的调和平均*

* F1 = 2 / ((1/precision) + (1/recall))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_train_5, y_pred))
print(recall_score(y_train_5, y_pred))
print(f1_score(y_train_5, y_pred))

#### 4. 准确率/召回率之间的折衷

In [ ]:
# 调用分类器的decision_function()方法, 自己选择阈值
y_scores = sgd_clf.decision_function([some_digit])
print(y_scores)
threshold = 0
y_some_digit_pred = (y_scores > threshold)
print(y_some_digit_pred)

In [ ]:
# 如何选择阈值
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                            mothod='decision_function')
# 对于任意可能的阈值，调用函数计算准确率与召回率
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

# 画出准确率，召回率与阈值的函数曲线
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precision[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recall[:-1], 'g-', label='Recall')
    plt.xlabel('Threshold')
    plt.legend(loc=;upper left)
    plt.ylim([0, 1])
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
# 画出 准确率vs召回率 曲线
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recall[:-1], precision[:-1])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
plot_precision_vs_recall(precisions, recalls)
plt.show()

In [ ]:
# 假设决定达到90%的准确率
y_train_pred_90 = (y_scores > 70000)
print(precision_score(y_train_5, y_train_pred_90))
print(recall_score(y_train_5, y_train_pred_90))

#### 5. ROC曲线

*TPR：真正例率，召回率*
* TPR = TP / (TP + FN)

*FPR：假正例率，反例被错误分成正例的比率*
* FPR = FP / (FP + TN)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, threshold = roc_curve(y_train_5, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1], [0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
# 计算ROC曲线下面的面积AUC比较性能的优劣
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_scores)

### RandomForestClassifier 比较 SGDClassifier

* RandomForestClassifier不提供decision_function()方法，提供predict_proba()方法

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                   method='predict_proba')

# 使用正例的概率当作样例的分数
y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

# 画ROC曲线
plt.plot(fpr, tpr, 'b:', label='SGD')
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc='bottom right')
plt.show()

### 多类分类